In [21]:
import requests
import gzip
import pandas as pd
import shutil
import datetime

In [22]:
ftpstream = requests.get("https://smn.conagua.gob.mx/tools/GUI/webservices/?method=3", headers={'User-Agent': 'Mozilla/5.0'})

In [23]:
type(ftpstream.content)

bytes

In [24]:
with open('./file.gz', 'wb') as file:
    file.write(ftpstream.content)

In [25]:
with gzip.open('./file.gz', 'rb') as f_in:
    with open('response.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [26]:
df = pd.read_json('./response.json')

In [27]:
df

,desciel,dh,dirvienc,dirvieng,dpt,dsem,hloc,hr,ides,idmun,lat,lon,nes,nhor,nmun,prec,probprec,raf,temp,velvien
0,Cielo cubierto,6,Noreste,37.31,13.99,Miércoles,20230823T19,87.16,20,54,17.3884,-97.2290,Oaxaca,-1,Magdalena Zahuatlán,0.17,95.0,17.65,14.91,11.41
1,Cielo cubierto,6,Noreste,39.41,13.99,Miércoles,20230823T20,90.64,20,54,17.3884,-97.2290,Oaxaca,0,Magdalena Zahuatlán,0.18,95.0,14.88,14.65,9.16
2,Cielo cubierto,6,Noreste,48.06,16.17,Miércoles,20230823T21,94.04,20,54,17.3884,-97.2290,Oaxaca,1,Magdalena Zahuatlán,0.18,95.0,11.33,16.59,8.74
3,Cielo cubierto,6,Noreste,67.49,17.64,Miércoles,20230823T22,95.20,20,54,17.3884,-97.2290,Oaxaca,2,Magdalena Zahuatlán,0.13,95.0,10.07,17.98,8.23
4,Cielo cubierto,6,Noreste,30.73,18.28,Miércoles,20230823T23,94.67,20,54,17.3884,-97.2290,Oaxaca,3,Magdalena Zahuatlán,0.07,95.0,10.47,18.66,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285703,Cielo cubierto,6,Este,73.44,21.83,Lunes,20230828T10,52.73,19,46,25.7553,-100.2896,Nuevo León,110,San Nicolás de los Garza,0.00,0.0,7.40,26.47,6.10
285704,Cielo cubierto,6,Este,78.83,22.25,Lunes,20230828T11,43.76,19,46,25.7553,-100.2896,Nuevo León,111,San Nicolás de los Garza,0.00,0.0,9.58,28.30,7.36
285705,Cielo cubierto,6,Este,85.92,22.59,Lunes,20230828T12,35.64,19,46,25.7553,-100.2896,Nuevo León,112,San Nicolás de los Garza,0.00,0.0,12.40,30.20,9.46
285706,Cielo cubierto,6,Este,95.46,23.37,Lunes,20230828T13,33.03,19,46,25.7553,-100.2896,Nuevo León,113,San Nicolás de los Garza,0.01,0.0,15.38,31.61,12.10


In [28]:
df[['fecha', 'hora']] = df['hloc'].str.split('T', expand=True)
df['fecha'] = pd.to_datetime(df['fecha'])

In [29]:
df['hora'] = df['hora'].astype('int')

In [30]:
df['fecha'].max()

Timestamp('2023-08-28 00:00:00')

In [31]:
df_2 = df[(df['fecha'] == datetime.date.today().isoformat()) & 
   (abs(df['hora'] - datetime.datetime.today().hour) < 3) & 
   (df['hora'] <= datetime.datetime.today().hour)].groupby(['ides', 'idmun']).mean()[['temp', 'prec']]


In [32]:
df_2 = df_2.reset_index()

In [33]:
df_2

,ides,idmun,temp,prec
0,1,1,21.066667,0.0
1,1,2,20.183333,0.0
2,1,3,20.226667,0.0
3,1,4,20.360000,0.0
4,1,5,19.850000,0.0
...,...,...,...,...
2458,32,54,19.446667,0.0
2459,32,55,18.693333,0.0
2460,32,56,20.223333,0.0
2461,32,57,21.660000,0.0
